<a href="https://colab.research.google.com/github/bso613/big_data_challenege/blob/main/s3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [98]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/graph

In [99]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-02-19 16:56:06--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.49MB/s    in 0.2s    

2022-02-19 16:56:06 (5.49 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [100]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("LoadingVGReview").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [101]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep=r"\t", header=True)

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [102]:
df.count()

1785997

In [103]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [104]:
review_table_df = df.select(['review_id','customer_id', 'product_id', 'product_parent', 'review_date'])
review_table_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RTIS3L2M1F5SM|   12039526|B001CXYMFS|     737716809| 2015-08-31|
| R1ZV7R40OLHKD|    9636577|B00M920ND6|     569686175| 2015-08-31|
|R3BH071QLH8QMC|    2331478|B0029CSOD2|      98937668| 2015-08-31|
|R127K9NTSXA2YH|   52495923|B00GOOSV98|      23143350| 2015-08-31|
|R32ZWUXDJPW27Q|   14533949|B00Y074JOM|     821342511| 2015-08-31|
|R3AQQ4YUKJWBA6|    2377552|B002UBI6W6|     328764615| 2015-08-31|
|R2F0POU5K6F73F|   17521011|B008XHCLFO|      24234603| 2015-08-31|
|R3VNR804HYSMR6|   19676307|B00BRA9R6A|     682267517| 2015-08-31|
| R3GZTM72WA2QH|     224068|B009EPWJLA|     435241890| 2015-08-31|
| RNQOY62705W1K|   48467989|B0000AV7GB|     256572651| 2015-08-31|
|R1VTIA3JTYBY02|     106569|B00008KTNN|     384411423| 2015-08-31|
|R29DOU8791QZL8|   48269642|B000A3IA0Y|     472622859| 2015-08

In [105]:
# Convert review_date column to date type
from pyspark.sql.functions import to_date
review_table_df = review_table_df.withColumn('review_date', to_date('review_date',"yyyy-MM-dd").alias('review_date'))
review_table_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RTIS3L2M1F5SM|   12039526|B001CXYMFS|     737716809| 2015-08-31|
| R1ZV7R40OLHKD|    9636577|B00M920ND6|     569686175| 2015-08-31|
|R3BH071QLH8QMC|    2331478|B0029CSOD2|      98937668| 2015-08-31|
|R127K9NTSXA2YH|   52495923|B00GOOSV98|      23143350| 2015-08-31|
|R32ZWUXDJPW27Q|   14533949|B00Y074JOM|     821342511| 2015-08-31|
|R3AQQ4YUKJWBA6|    2377552|B002UBI6W6|     328764615| 2015-08-31|
|R2F0POU5K6F73F|   17521011|B008XHCLFO|      24234603| 2015-08-31|
|R3VNR804HYSMR6|   19676307|B00BRA9R6A|     682267517| 2015-08-31|
| R3GZTM72WA2QH|     224068|B009EPWJLA|     435241890| 2015-08-31|
| RNQOY62705W1K|   48467989|B0000AV7GB|     256572651| 2015-08-31|
|R1VTIA3JTYBY02|     106569|B00008KTNN|     384411423| 2015-08-31|
|R29DOU8791QZL8|   48269642|B000A3IA0Y|     472622859| 2015-08

In [106]:
# Check for duplicates
if review_table_df.count() > review_table_df.dropDuplicates(['review_id','customer_id','product_id','product_parent','review_date']).count():
    raise ValueError('Data has duplicates')

In [107]:
review_table_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- review_date: date (nullable = true)



In [112]:
# change type of star_rating from string to integer
review_table_df = review_table_df.withColumn('customer_id', col('customer_id').cast('integer'))
review_table_df = review_table_df.withColumn('product_parent', col('product_parent').cast('integer'))
review_table_df.printSchema()
review_table_df.show()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RTIS3L2M1F5SM|   12039526|B001CXYMFS|     737716809| 2015-08-31|
| R1ZV7R40OLHKD|    9636577|B00M920ND6|     569686175| 2015-08-31|
|R3BH071QLH8QMC|    2331478|B0029CSOD2|      98937668| 2015-08-31|
|R127K9NTSXA2YH|   52495923|B00GOOSV98|      23143350| 2015-08-31|
|R32ZWUXDJPW27Q|   14533949|B00Y074JOM|     821342511| 2015-08-31|
|R3AQQ4YUKJWBA6|    2377552|B002UBI6W6|     328764615| 2015-08-31|
|R2F0POU5K6F73F|   17521011|B008XHCLFO|      24234603| 2015-08-31|
|R3VNR804HYSMR6|   19676307|B00BRA9R6A|     682267517| 2015-08-31|
| R3GZTM72WA2QH|     224068|B009EPWJLA|   

In [113]:
customer_id_count = review_table_df.select(['customer_id'])
customer_id_count.show()

+-----------+
|customer_id|
+-----------+
|   12039526|
|    9636577|
|    2331478|
|   52495923|
|   14533949|
|    2377552|
|   17521011|
|   19676307|
|     224068|
|   48467989|
|     106569|
|   48269642|
|   52738710|
|   10556786|
|    2963837|
|   23092109|
|   23091728|
|   10712640|
|   17455376|
|   14754850|
+-----------+
only showing top 20 rows



In [114]:
customer_id_count = customer_id_count.groupby('customer_id').count()
customer_id_count.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   48670265|    1|
|   49103216|    2|
|    1131200|    1|
|   43076447|    2|
|   46261368|    1|
|    4883305|    5|
|   41192649|    1|
|   40985731|    7|
|   10437900|    2|
|   22245671|    1|
|    2574873|    1|
|    4696154|    1|
|    5621202|    1|
|    5871933|    2|
|   44089812|    1|
|    2845910|    1|
|    5274369|    1|
|   39069693|    2|
|     137793|    1|
|   31914942|    3|
+-----------+-----+
only showing top 20 rows



In [115]:
customer_df = customer_id_count.select(['customer_id', 'count'])
customer_df.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   48670265|    1|
|   49103216|    2|
|    1131200|    1|
|   43076447|    2|
|   46261368|    1|
|    4883305|    5|
|   41192649|    1|
|   40985731|    7|
|   10437900|    2|
|   22245671|    1|
|    2574873|    1|
|    4696154|    1|
|    5621202|    1|
|    5871933|    2|
|   44089812|    1|
|    2845910|    1|
|    5274369|    1|
|   39069693|    2|
|     137793|    1|
|   31914942|    3|
+-----------+-----+
only showing top 20 rows



In [116]:
# rename column to correct name
customer_df = customer_df.withColumnRenamed('count', 'customer_count')

In [117]:
# Check for duplicates
if customer_df.count() > customer_df.dropDuplicates(['customer_id']).count():
    raise ValueError('Data has duplicates')

In [118]:
# convert customer_count from long to integer
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

customer_df = customer_df.withColumn('customer_count', col('customer_count').cast('integer'))
customer_df.printSchema()
customer_df.show()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   48670265|             1|
|   49103216|             2|
|    1131200|             1|
|   43076447|             2|
|   46261368|             1|
|    4883305|             5|
|   41192649|             1|
|   40985731|             7|
|   10437900|             2|
|   22245671|             1|
|    2574873|             1|
|    4696154|             1|
|    5621202|             1|
|    5871933|             2|
|   44089812|             1|
|    2845910|             1|
|    5274369|             1|
|   39069693|             2|
|     137793|             1|
|   31914942|             3|
+-----------+--------------+
only showing top 20 rows



In [119]:
customer_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [120]:
product_df = df.select(['product_id', 'product_title'])
product_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B001CXYMFS|Thrustmaster T-Fl...|
|B00M920ND6|Tonsee 6 buttons ...|
|B0029CSOD2|Hidden Mysteries:...|
|B00GOOSV98|GelTabz Performan...|
|B00Y074JOM|Zero Suit Samus a...|
|B002UBI6W6|Psyclone Recharge...|
|B008XHCLFO|Protection for yo...|
|B00BRA9R6A|   Nerf 3DS XL Armor|
|B009EPWJLA|One Piece: Pirate...|
|B0000AV7GB|Playstation 2 Dan...|
|B00008KTNN|Metal Arms: Glitc...|
|B000A3IA0Y|72 Pin Connector ...|
|B0053BQN34|uDraw Gametablet ...|
|B002I0HIMI|NBA 2K12(Covers M...|
|B0081EH12M|New Trigger Grips...|
|B005FMLZQQ|Xbox 360 Media Re...|
|B002BSA388|Super Mario Galaxy 2|
|B00BUSLSAC|Nintendo 3DS XL -...|
|B00KWF38AW|Captain Toad:  Tr...|
|B00BRQJYA8|Lego Batman 2: DC...|
+----------+--------------------+
only showing top 20 rows



In [121]:
# Check for duplicates
if product_df.count() > product_df.dropDuplicates(['product_id']).count():
    raise ValueError('Data has duplicates')

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/content/spark-3.2.1-bin-hadoop2.7/python/lib/py4j-0.10.9.3-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/content/spark-3.2.1-bin-hadoop2.7/python/lib/py4j-0.10.9.3-src.zip/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [122]:
# drop duplicates of product id (is a unique primary key) and see new row count
product_df = product_df.dropDuplicates(['product_id'])
product_df.count()

65792

In [123]:
product_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [124]:
vine_df = df.select('review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine')
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RTIS3L2M1F5SM|          5|            0|          0|   N|
| R1ZV7R40OLHKD|          5|            0|          0|   N|
|R3BH071QLH8QMC|          1|            0|          1|   N|
|R127K9NTSXA2YH|          3|            0|          0|   N|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|
|R2F0POU5K6F73F|          5|            0|          0|   N|
|R3VNR804HYSMR6|          5|            0|          0|   N|
| R3GZTM72WA2QH|          5|            0|          0|   N|
| RNQOY62705W1K|          4|            0|          0|   N|
|R1VTIA3JTYBY02|          5|            0|          0|   N|
|R29DOU8791QZL8|          1|            0|          0|   N|
|R15DUT1VIJ9RJZ|          2|            0|          0|   N|
|R3IMF2MQ3OU9ZM|          4|            

In [127]:
# change type of star_rating from string to integer
vine_df = vine_df.withColumn('star_rating', col('star_rating').cast('integer'))
vine_df = vine_df.withColumn('helpful_votes', col('helpful_votes').cast('integer'))
vine_df = vine_df.withColumn('total_votes', col('total_votes').cast('integer'))
vine_df.printSchema()
vine_df.show()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RTIS3L2M1F5SM|          5|            0|          0|   N|
| R1ZV7R40OLHKD|          5|            0|          0|   N|
|R3BH071QLH8QMC|          1|            0|          1|   N|
|R127K9NTSXA2YH|          3|            0|          0|   N|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|
|R2F0POU5K6F73F|          5|            0|          0|   N|
|R3VNR804HYSMR6|          5|            0|          0|   N|
| R3GZTM72WA2QH|          5|            0|          0|   N|
| RNQOY62705W1K|          4|            0|          0|   N|
|R1

In [128]:
vine_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [134]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://big-data-challenege.cfeadpixshb5.us-east-1.rds.amazonaws.com:5432/big-data-challenge"
config = {"user":"postgres",
          "password": "Allstar123",
          "driver":"org.postgresql.Driver"}

In [135]:
# Write DataFrame to table

review_table_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)
customer_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config) # first data set was uploaded so skipping this
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

Py4JJavaError: ignored